In [1]:
import sys
if (path := "C:/Users/Tom/PycharmProjects/python-hvac") not in sys.path:
    sys.path.append(path)

In [2]:
from hvac import Quantity, print_doc_string

Q_ = Quantity

# Chapter 5: Modeling of a Thermal Zone
---

In the previous four notebooks some main objects were already introduced that will be used to model a thermal zone. Besides exterior building elements and windows, a thermal zone can also be bounded by interior building elements, with or without interior doors, that separate a thermal zone from adjacent thermal zones in a building. Also, there are still a few other things that are good to know when modeling the thermal zone of a building.

## Interior Building Elements

Interior building elements are modeled by the `InteriorBuildingElement` class. Like exterior building elements, their size and construction are also characterized by their gross area and the construction assembly they're made of. They also have a parameter `F_rad` to split the conduction heat gain through the interior building element into a convective and a radiative fraction. Where `ExteriorBuildingElement` objects carry data about the outdoor environment (the `WeatherData` object), an `InteriorBuildingElement` object needs to carry data about the adjacent thermal zone it separates from the thermal zone under consideration. These data are the zone-air temperature in the adjacent zone (which is considered to be a fixed value), and, in case ventilation air is transferred from the adjacent zone to the considered zone, the (design value of the) transfer air volume flow rate.   

In [3]:
from hvac.cooling_load_calc import InteriorBuildingElement

In [4]:
print_doc_string(InteriorBuildingElement.create)

Creates an `InteriorBuildingElement` object.

Parameters
----------
name:
    Name to identify the interior building element.
adjacent_zone_name:
    Name given to the adjacent zone.
constr_assem:
    The construction assembly the interior building element is made of.
gross_area:
    The gross surface area of the interior building element, i.e.
    including any large openings such as doors and windows.
adjacent_zone_temperature:
    The zone-air temperature on the other side of the interior building
    element (i.e. the zone-air temperature of the adjacent zone).
V_dot_trf: optional
    Volume flow rate of air transferred from the adjacent zone. If 
    `None`, no air is transferred. 
F_rad: default 0.46
    Radiative fraction of the conduction heat gain through the interior
    building element, taken up by the interior thermal mass of the
    space. (see ASHRAE Fundamentals 2017, chapter 18, table 14).

Returns
-------
`InteriorBuildingElement` object.

Parameter Types & Defaults
-

Like exterior building elements, (interior) doors can be added to an interior building element with the method `add_door(...)`.

In [5]:
print_doc_string(InteriorBuildingElement.add_door)

Adds a door into the interior building element.

An interior door is also represented by an `InteriorBuildingElement` 
object.

Parameters
----------
name:
    Identifies the door.
    Doors are hold in a dictionary `self.doors`. The ID of the door is
    used as the key in the dictionary.
width:
    The width of the door.
height:
    The height of the door.
constr_assem:
    The construction assembly the door is made off.
F_rad: default 0.46
    Radiative fraction of the conduction heat gain through the door,
    taken up by the interior thermal mass of the space. (see ASHRAE
    Fundamentals 2017, chapter 18, table 14).

Parameter Types & Defaults
--------------------------

name: 'str'
width: 'Quantity'
height: 'Quantity'
constr_assem: 'ConstructionAssembly'
F_rad: 'float' = 0.46


## Simple Construction Assemblies

In the model of a thermal zone, exterior doors, interior building elements (ceilings, interior walls), and interior doors don't really need to have the detailed layered composition that is used for thick exterior building elements (roofs, exterior walls). Construction assemblies can also be configured without construction layers, knowing only their unit transmittance or unit thermal resistance. For example, if only the unit transmittance of an interior door is known, we can define the construction assembly of the interior door as follows:

In [6]:
from hvac.cooling_load_calc import ConstructionAssembly

int_door_assem = ConstructionAssembly.create(
    ID='int-door-assem',
    layers=None,
    U=Q_(4.0, 'W / (m**2 * K)')
)

If we don't specify the geometry (area and thickness) of the construction assembly, it is given default values.   

In [7]:
print(int_door_assem.area, int_door_assem.thickness, sep='\n')

1.0 meter ** 2
0.0 meter


With this construction assembly, we could then create an interior door as follows:

In [8]:
int_door = InteriorBuildingElement.create(
    name='int-door',
    adjacent_zone_name='zone-2',
    constr_assem=int_door_assem,
    gross_area=Q_(80, 'cm') * Q_(220, 'cm'),
    adjacent_zone_temperature=Q_(24, 'degC')
)

Interior building elements -but exterior building elements and windows also- have a property `UA` which returns the steady-state overall transmittance of the element. Our interior door has *UA*-value of:

In [9]:
print(int_door.UA)

7.04 watt / kelvin


In a thermal zone model, conduction heat gains through interior building elements, including interior doors, through windows, and through exterior doors will be calculated based on this steady-state UA-value (i.e., the thermal inertia of these elements is ignored in the model of a thermal zone).

## Heterogeneous Construction Layers

So far the construction assemblies were composed of homogeneous construction layers (i.e. each layer in the assembly is made of only a single material). However, a wooden stud wall for example, is made of two materials: wooden studs with air spaces or insulation in between. From a thermal viewpoint, this heterogeneous construction layer can be replaced by an equivalent homogeneous construction layer. The thermal resistance of the wooden studs and the thermal resistance of the insulation in between the studs act like thermal resistors connected in parallel (their area-weighted thermal conductances add up) and also their area-weighted thermal capacities add up.

In the code block below it is demonstrated how the equivalent construction layer of a wooden stud wall is programmed. Wooden studs are placed every 60 cm. The studs are 5 cm wide and 15 cm deep. A unit wall height of 1 m is considered.

In [10]:
from hvac.cooling_load_calc import Material, SolidLayer, Geometry

wood_stud = SolidLayer.create(
    ID='wood_stud',
    geometry=Geometry(t=Q_(15, 'cm'), w=Q_(5, 'cm'), h=Q_(1, 'm')),
    material=Material(
        k=Q_(0.10, 'W / (m * K)'),
        rho=Q_(550, 'kg / m**3'),
        c=Q_(1630, 'J / (kg * K)')
    ),
    num_slices=3
)

print(wood_stud.R.to('K * m**2 / W'))
print(wood_stud.geometry.A.to('m**2'))
wood_stud_R_tot = wood_stud.R / wood_stud.geometry.A
print(wood_stud_R_tot.to('K / W'))
print()

insulation = SolidLayer.create(
    ID='insulation',
    geometry=Geometry(t=Q_(15, 'cm'), w=Q_(55, 'cm'), h=Q_(1, 'm')),
    material=Material(
        k=Q_(0.038, 'W / (m * K)'),
        rho=Q_(22, 'kg / m**3'),
        c=Q_(1030, 'J / (kg * K)')
    ),
    num_slices=3
)

print(insulation.R.to('K * m**2 / W'))
print(insulation.geometry.A.to('m**2'))
insulation_R_tot = insulation.R / insulation.geometry.A
print(insulation_R_tot.to('K / W'))
print()

# Combining the two parallel solid construction layers creates a
# new construction layer
stud_wall = wood_stud // insulation

print(type(stud_wall))
print(stud_wall.R.to('K * m**2 / W'))
print(stud_wall.geometry.A.to('m**2'))
stud_wall_R_tot = stud_wall.R / stud_wall.geometry.A
print(stud_wall_R_tot.to('K / W'))

1.5 kelvin * meter ** 2 / watt
0.05 meter ** 2
30.0 kelvin / watt

3.947368421052632 kelvin * meter ** 2 / watt
0.55 meter ** 2
7.177033492822967 kelvin / watt

<class 'hvac.cooling_load_calc.building.construction_assembly.ConstructionLayer'>
3.474903474903475 kelvin * meter ** 2 / watt
0.6 meter ** 2
5.791505791505792 kelvin / watt


> **Note**<br>
Two `SolidLayer` objects were combined into a new object of class `ConstructionLayer`, which is the base class of class `SolidLayer` and the other types of construction layers (class `AirLayer` and class `SurfaceFilm`). It is also possible to combine a `SolidLayer` object with an `AirLayer` object. Objects of class `ConstructionAssembly`, that contain the construction layers, don't need to know about the exact type of a construction layer; they only work with the attributes and methods of the base class `ConstructionLayer`, which are shared with the derived classes.  

## Construction Data Shelves

To avoid that the same construction material, the same construction assembly, or the same window thermal properties must be redefined over and over again, shelves are provided to store these objects locally on disk. The use of a shelf has already been demonstrated for construction assemblies in notebook no. 1 and for window thermal properties in notebook no. 3.

But the `cooling_load_calc` package also contains a subpackage `construction_data` with a number of dedicated "shelf classes":
- class `MaterialShelf` to store building construction materials,
- class `ConstructionAssemblyShelf` to store construction assemblies,
- class `WindowsPropertiesShelf` to store thermal properties of window glazing systems.

Also inside subpackage `construction_data` is a subpackage `wtcb`. This subpackage has functions to create a number of predefined construction assemblies. These functions are grouped in different "catalog classes":
- `ExteriorWallCatalog`, 
- `FloorCatalog`,
- `InteriorWallCatalog`,
- `CeilingCatalog`, and
- `RoofCatalog`.

The names (IDs) of the construction assemblies refer to the sheets in a catalog of commonly used construction assemblies published by the Belgian institute *WTCB* (nowadays *Buildwise*). This catalog (in PDF-format) can be found in the *docs* folder of this repository.

Below it is demonstrated how the predefined construction assembly for an exterior wall can be fetched from the `ExteriorWallCatalog`.

In [11]:
from hvac.cooling_load_calc.construction_data import ExteriorWallCatalog

In [12]:
print_doc_string(ExteriorWallCatalog.__init__)

Creates an instance of `ExteriorWallCatalog`.

Parameters
----------
t_ins:
    Default thickness of the insulation layer.
T_ext:
    Default design value for outdoor air temperature.
T_int:
    Default design value for indoor air temperature.
v_wind:
    Default design value for wind speed.

Parameter Types & Defaults
--------------------------

t_ins: pint.Quantity = <Quantity(10, 'centimeter')>
T_ext: pint.Quantity = <Quantity(0, 'degree_Celsius')>
T_int: pint.Quantity = <Quantity(20, 'degree_Celsius')>
v_wind: pint.Quantity = <Quantity(4, 'meter / second')>


On instantiation of the class, some parameters can be assigned default values: insulation thickness, outdoor air temperature, indoor air temperature, and wind speed. However, these default values can be overridden each time the `ExteriorWallCatalog` object is called to create a construction assembly. 

In [13]:
ext_wall_catalog = ExteriorWallCatalog(
    t_ins=Q_(18, 'cm'),
    T_ext=Q_(26, 'degC'),
    T_int=Q_(22, 'degC'),
    v_wind=Q_(3, 'm /s')
)

Let us retrieve the construction assembly for the exterior wall with ID *F7* in the catalog ("*fiche nr. 07, wandtype: Buitenwanden*", p. 15/60 of the PDF-catalog in folder *docs*). We will change the insulation thickness to 16 cm (instead of using the default value we have specified when we instantiated the catalog).

In [14]:
print_doc_string(ext_wall_catalog.__call__)

Creates the construction assembly of the exterior wall indicated by
`ID`. `ID` refers to the wall ID number in the WTCB catalog
(see /docs/wtcb_catalog/wtcb_catalog.pdf).

Parameters
----------
ID:
    Sheet number of the exterior wall in the WTCB catalog.
t_ins:
    Thickness of the insulation layer. Overrides the value assigned on
    instantiation of the `ExteriorWallCatalog` class.
T_ext:
    The outdoor air design temperature. Overrides the value assigned on
    instantiation of the `ExteriorWallCatalog` class.
T_int:
    The indoor air design temperature. Overrides the value assigned on
    instantiation of the `ExteriorWallCatalog` class.
v_wind:
    Design value for the wind speed. Overrides the value assigned on
    instantiation of the `ExteriorWallCatalog` class.

Parameter Types & Defaults
--------------------------
ID: str
t_ins: pint.Quantity | None = None
T_ext: pint.Quantity | None = None
T_int: pint.Quantity | None = None
v_wind: pint.Quantity | None = None


> **Note**<br>
> Before any of the catalogs can be used, the shelf with building materials (and the shelf with window thermal
properties) must have been installed first. By default these shelves will be created in a directory `wtcb-database` in the user's home directory (see `wtcb.setup.py`). To create these shelves, the scripts `materials.py` and `windows.py` in the `wtcb` package need to be executed.

In [15]:
try:
    ext_wall_assem_F7 = ext_wall_catalog('F7', t_ins=Q_(16, 'cm'))
except KeyError:
    # this will create the "wtcb-database" folder in the user's home 
    # directory and install the materials and windows shelves
    from hvac.cooling_load_calc.construction_data.wtcb import materials
    from hvac.cooling_load_calc.construction_data.wtcb import windows
    materials.main(show=False)
    windows.main(show=False)
    ext_wall_assem_F7 = ext_wall_catalog('F7', t_ins=Q_(16, 'cm'))

print(ext_wall_assem_F7)

construction assembly: ext-wall-wtcb-F7
SurfaceFilm 'ext_surf_film': t = 0.000 m, R = 0.05 K·m²/W, C = 0.00 J/K/m², nodes = 1
SolidLayer 'outer_leaf': t = 0.100 m, R = 0.06 K·m²/W, C = 126000.00 J/K/m², nodes = 10
AirLayer 'air_space': t = 0.050 m, R = 0.07 K·m²/W, C = 0.00 J/K/m², nodes = 1
SolidLayer 'insulation': t = 0.160 m, R = 3.54 K·m²/W, C = 35280.00 J/K/m², nodes = 5
SolidLayer 'inner_leaf': t = 0.140 m, R = 0.30 K·m²/W, C = 168000.00 J/K/m², nodes = 15
SolidLayer 'gypsum_layer': t = 0.015 m, R = 0.03 K·m²/W, C = 16380.00 J/K/m², nodes = 1
SurfaceFilm 'int_surf_film': t = 0.000 m, R = 0.13 K·m²/W, C = 0.00 J/K/m², nodes = 1



## Creating a Thermal Zone

A thermal zone is modeled by the `ThermalZone` class. To create a `ThermalZone` object, the class method `create(...)` is called.

In [16]:
from hvac.cooling_load_calc import ThermalZone

In [17]:
print_doc_string(ThermalZone.create)

Creates a `Zone` object.

Parameters
----------
name:
    Name of the temperature zone.
floor_area:
    Floor area of the zone.
ceiling_height:
    Ceiling height in the zone.
unit_R_im:
    Unit thermal resistance of the interior mass surface. 
unit_C_im:
    Unit thermal capacity of the interior mass, i.e. per unit area.
weather_data:
    Instance of class `WeatherData` encapsulating the climatic design
    data for the selected design day, which was given when the 
    `WeatherData` object was instantiated.
factor_surf_im:
    Multiplication factor for the interior mass surface area.

Keyword Arguments
-----------------
air_temperature:
    Reference zone air temperature used for calculating the thermal
    capacity of the zone air. Default value is 20 °C.
air_pressure:
    Reference zone air pressure used for calculating the thermal 
    capacity of the zone air. Default value is 101,325 Pa.
RH_zone:
    Setpoint or desired value of the relative air humidity in the zone.
    Defaul

Like an `ExteriorBuildingElement` object, a `ThermalZone` object is represented by a linear thermal network model. This model has two temperature nodes: a zone-air node, which represents the air volume in the thermal zone, and an interior mass node, which incorporates the internal structural mass (floor, ceiling, partitions) and also furnishings of the thermal zone, to take their thermal storage effect on the cooling/heating load of the thermal zone into account. The concept of this model is explained in more detail in *Mitchell, J. W., & Braun, J. E. (2012). Principles of Heating, Ventilation, and Air Conditioning in Buildings. John Wiley & Sons*.

The interior mass node and the zone air node are coupled by a thermal resistor `R_im`, i.e. the surface resistance of the interior mass. It is the sum of the convective heat transfer resistance and that of any floor surfaces such as carpeting. The unit convective resistance for the interior of buildings is around 0.12 K.m²/W. The unit thermal resistance of carpeting ranges from about 0.1 to 0.4 K.m²/W.

The capacitance of the interior mass is an aggregate value of capacitance that takes into account storage in the floor, ceilings, partitions, and other furnishings. The unit thermal capacity of the interior mass may range from 100 kJ/(m².K) for light building construction to 300 kJ/(m².K) for heavy building construction. The surface area also depends on the type of internal structure, and is at least equal to the floor area. A reasonable estimate would be two to three times the actual floor area.  

After the `ThermalZone` object has been instantiated, we can add the exterior and interior building elements that surround the zone, using the methods `add_ext_build_elems(*ext_build_elems)` and `add_int_build_elems(*int_build_elems)` respectively. Internal heat gains can be added to the thermal zone with the method `add_internal_heat_gains(*ihg: InternalHeatGain)`.

## Ventilation

To estimate ventilation heat gains in the cooling/heating load of the thermal zone, a "ventilation model" is used according to  *European standard EN 12831-1:2017 Energy performance of buildings - Method for calculation of the design heat load - Part 1: Space heating load, Module M3-3.*

In this ventilation model multiple spaces or thermal zones can be part of a single ventilation zone. The standard defines a ventilation zone as a group of rooms that are air-connected by design, either directly or indirectly, e.g., through internally mounted air transfer devices, shortened door leafs, etc. By design, there is no air transfer between ventilation zones.

To add ventilation to a thermal zone, a `VentilationZone` object must first be created. Then, the method `add_ventilation(...)` of the `ThermalZone` object can be called, passing the `VentilationZone` object into the call. 

In [18]:
from hvac.cooling_load_calc import VentilationZone

In [19]:
print_doc_string(VentilationZone.create)

Creates a `VentilationZone` object.

Parameters
----------
name:
    Identifies the ventilation zone.
q_env_50:
    Air permeability of the building envelope at a pressure difference
    of 50 Pa between the interior and exterior with any ATDs closed or
    sealed (see EN 12831-1 (2017), B.2.10).
dP_ATD_d:
    Design pressure difference of the ATDs in the zone (see EN 12831-1
    (2017), B.2.12).
v_leak:
    Pressure exponent for air leakages (see EN 12831-1 (2017), B.2.13).
f_fac:
    Adjustment factor for the number of wind exposed facades of the zone
    (see EN 12831-1 (2017), B.2.15). The default value applies to 1 wind
    exposed facade.
f_V:
    Coefficient for the volume flow ratio of the zone (see EN 12831-1
    (2017), B.2.11, Table B.8). The default value applies to more than
    1 external facade, height of the zone above ground level between 0
    and 50 m, normal shielding, and a zone height between 5 and 10 m.
f_dir:
    Factor for the orientation of the zone (see EN 12

In [20]:
print_doc_string(ThermalZone.add_ventilation)

Adds ventilation to the thermal zone.

Parameters
----------
ventilation_zone:
    The ventilation zone to which the thermal zone belongs. 
n_min:
    Minimum air change rate required for the space for reasons of air
    quality/hygiene and comfort (EN 12831-1, B.2.10 - Table B.7).
    The default value applies to permanent dwelling areas (living rooms,
    offices) and a ceiling height less than 3 m.
V_dot_open:
    External air volume flow rate into the space through large openings
    (EN 12831-1, Annex G).
V_dot_ATD_d:
    Design air volume flow rate of the ATDs in the room (EN 12831-1,
    B.2.12). Only relevant if ATDs (Air Terminal Devices) are used for
    ventilation (i.e., passive devices that allow air flow through a
    building element; it does not include the air out- or inlets of 
    fan-assisted ventilation systems).
V_dot_sup:
    Supply air volume flow rate from the ventilation system into the
    space.
V_dot_exh:
    Exhaust ventilation air volume flow rate from th

Where possible, reasonable default values were assigned to the call parameters of both methods, according to default values mentioned in the standard. For more detail about these parameters, standard EN 12831-1 should be consulted. 

## Example of Modeling a Thermal Zone

The Python module `thermal_zone_example.py` in the same folder as this notebook gives a quick demonstration of all the steps that need to be taken to program the model of a thermal zone (`ThermalZone` object). This model can then be used for simulating the heat flows to or from the thermal zone and the temperature in the zone. This is demonstrated in the following notebook no. 6.